<a href="https://colab.research.google.com/github/ericchchiu/testing/blob/master/20210530RoBERTaLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from https://towardsdatascience.com/transformers-retraining-roberta-base-using-the-roberta-mlm-procedure-7422160d5764
# 2020121 Tanmay Garg
# need to tidy up the code. The pretrained file roberta-retrained is no longer there 

In [ ]:
!pip install transformers 

     |████████████████████████████████| 2.3MB 30.8MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 
     |████████████████████████████████| 901kB 45.4MB/s 


In [ ]:
from transformers import pipeline

In [ ]:
fill_mask = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

[{'score': 0.16661563515663147,
  'sequence': 'Send these pictures back!',
  'token': 3493,
  'token_str': ' pictures'},
 {'score': 0.10792822390794754,
  'sequence': 'Send these photos back!',
  'token': 2356,
  'token_str': ' photos'},
 {'score': 0.07670954614877701,
  'sequence': 'Send these emails back!',
  'token': 5575,
  'token_str': ' emails'},
 {'score': 0.04860773682594299,
  'sequence': 'Send these images back!',
  'token': 3156,
  'token_str': ' images'},
 {'score': 0.048417724668979645,
  'sequence': 'Send these letters back!',
  'token': 5430,
  'token_str': ' letters'}]

In [ ]:
!git clone https://github.com/cardiffnlp/tweeteval /tmp/tweeteval

from transformers import RobertaTokenizer, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Cloning into '/tmp/tweeteval'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 344 (delta 7), reused 10 (delta 1), pack-reused 318
Receiving objects: 100% (344/344), 10.78 MiB | 18.81 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/tmp/tweeteval/datasets/hate/train_text.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=48,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,2.338900
1000,2.083500
1500,1.938200
2000,1.833700
2500,1.737200
3000,1.642300
3500,1.580800
4000,1.520000
4500,1.482600


TrainOutput(global_step=4700, training_loss=1.781688037628823, metrics={'train_runtime': 4694.3254, 'train_samples_per_second': 1.001, 'total_flos': 559242548728668.0, 'epoch': 25.0, 'init_mem_cpu_alloc_delta': 2153218048, 'init_mem_gpu_alloc_delta': 500087808, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -212135936, 'train_mem_gpu_alloc_delta': 1562469376, 'train_mem_cpu_peaked_delta': 222605312, 'train_mem_gpu_peaked_delta': 9978781184})

In [ ]:
trainer.save_model("./roberta-retrained")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    #model="/content/drive/MyDrive/roberta-retrained",
    model="./roberta-retrained",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

[{'score': 0.12151338160037994,
  'sequence': 'Send these people back!',
  'token': 82,
  'token_str': ' people'},
 {'score': 0.09814539551734924,
  'sequence': 'Send these idiots back!',
  'token': 39984,
  'token_str': ' idiots'},
 {'score': 0.08012119680643082,
  'sequence': 'Send these refugees back!',
  'token': 4498,
  'token_str': ' refugees'},
 {'score': 0.0684138834476471,
  'sequence': 'Send these guys back!',
  'token': 1669,
  'token_str': ' guys'},
 {'score': 0.061683546751737595,
  'sequence': 'Send these kids back!',
  'token': 1159,
  'token_str': ' kids'}]

In [ ]:
# below three cells are for copying the file roberta-retrained to "drive/My Drive/""

In [ ]:
# from google.colab import drive
# drive.mount('drive')

Mounted at drive


In [ ]:
# !cp -r roberta-retrained "drive/My Drive/"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')